# 1. Backfill Pipeline

## 1.1. Setup

### 1.1.1. Import Libraries

In [ ]:
# Standard imports
import os
from pathlib import Path
import sys
import json
import time
from datetime import date, datetime, timedelta
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore", module="IPython")

#  Establish project root directory
def find_project_root(start: Path):
    for parent in [start] + list(start.parents):
        if (parent / "pyproject.toml").exists():
            return parent
    return start

root_dir = find_project_root(Path().absolute())
print("Project root dir:", root_dir)

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# Third-party imports
import requests
import pandas as pd
import numpy as np
import great_expectations as gx
import hopsworks
from urllib3.exceptions import ProtocolError
from requests.exceptions import ConnectionError, Timeout, RequestException
from confluent_kafka import KafkaException
from hsfs.client.exceptions import RestAPIError
from collections import defaultdict
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
from scipy.spatial.distance import cdist

#  Project imports
from utils import cleaning, config, feature_engineering, fetchers, hopsworks_admin, incremental, metadata, visualization

today = datetime.today().date()

### 1.1.2. Load settings and Initialize Hopsworks Connection

In [ ]:

def detect_environment():
    if (
        "HOPSWORKS_JOB_ID" in os.environ
        or "HOPSWORKS_PROJECT_ID" in os.environ
        or "HOPSWORKS_JOB_NAME" in os.environ
    ):
        return "job"

    cwd = os.getcwd()
    if cwd.startswith("/hopsfs/Jupyter"):
        return "jupyter"

    return "local"

env = detect_environment()
print(f"Detected environment: {env}")

# Load secrets based on environment

if env in ("job", "jupyter"):
    project = hopsworks.login()
    secrets_api = hopsworks.get_secrets_api()

    for key in ["HOPSWORKS_API_KEY", "AQICN_API_KEY", "GH_PAT", "GH_USERNAME"]:
        os.environ[key] = secrets_api.get_secret(key).value

else:
    load_dotenv()

# Load Pydantic settings

settings = config.HopsworksSettings()

HOPSWORKS_API_KEY = settings.HOPSWORKS_API_KEY.get_secret_value()
AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()
GITHUB_USERNAME = settings.GH_USERNAME.get_secret_value()

# Login to Hopsworks using the API key

project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

print("Environment initialized and Hopsworks connected!")
print(project.name)


### 1.1.3. Repository management

In [ ]:
repo_dir = hopsworks_admin.clone_or_update_repo(GITHUB_USERNAME)
os.chdir(repo_dir)

### 1.1.4. Configure API Keys and Secrets

In [ ]:
secrets = hopsworks.get_secrets_api()

try:
    secrets.get_secret("AQICN_API_KEY")
except:
    secrets.create_secret("AQICN_API_KEY", settings.AQICN_API_KEY.get_secret_value())

## 1.2. Create Feature Groups

In [ ]:
# # Clean Up Old Resources (if recreating)

# # Delete feature view first (blocks feature group deletion)
# print("🗑️ Step 1: Delete feature view (if exists)...")
# try:
#     hopsworks_admin.delete_feature_views(fs, "air_quality_complete_fv")
# except Exception as e:
#     print(f"   Note: {e}")

# # Delete old feature groups
# print("\n🗑️ Step 2: Delete old feature groups (if exist)...")
# try:
#     hopsworks_admin.delete_feature_groups(fs, "air_quality")
# except Exception as e:
#     print(f"   Note: {e}")

# try:
#     hopsworks_admin.delete_feature_groups(fs, "weather")
# except Exception as e:
#     print(f"   Note: {e}")

# print("\n✅ Cleanup complete - ready to create fresh feature groups")

In [ ]:
air_quality_fg, weather_fg = hopsworks_admin.create_feature_groups(fs)

## 1.3. Check what backfill is needed

In [ ]:
data_dir = os.path.join(root_dir, "data")
dir_list = os.listdir(data_dir)

sensor_locations = metadata.get_sensor_locations_dict(air_quality_fg)
existing_sensors = set(sensor_locations.keys())

print(f"📋 Found {len(existing_sensors)} sensors already in feature store")
print(f"📍 Loaded locations for {len(sensor_locations)} existing sensors")

total_sensors = len([f for f in dir_list if f.endswith(".csv")])
remaining = total_sensors - len(existing_sensors)
print(f"📊 Total sensors: {total_sensors}, Already processed: {len(existing_sensors)}, Remaining: {remaining}")

## 1.4. Backfill
When performed for the first time, might take a long time if many added sensors.

In [ ]:

if total_sensors != len(existing_sensors):
    print("\n🚀 Starting backfill process...\n")
    # Track processing stats
    successful = 0
    failed = 0
    skipped = 0
    failed_sensors = []  # Track which sensors failed and why

    for file in dir_list:
        if not file.endswith(".csv"):
            continue

        file_path = os.path.join(data_dir, file)
        
        try:
            aq_df_raw, street, city, country, feed_url, sensor_id = metadata.read_sensor_data(
                file_path, AQICN_API_KEY
            )

            sensor_id = int(sensor_id)

            # Skip if already processed
            if sensor_id in existing_sensors:
                skipped += 1
                continue

            # Get working feed URL using sensor ID and API token
            try:
                working_feed_url = fetchers.get_working_feed_url(sensor_id, AQICN_API_KEY)
            except Exception as url_err:
                print(f"⚠️ Sensor {sensor_id}: Could not resolve feed URL - {url_err}")
                working_feed_url = feed_url  # Fallback to CSV feed_url if resolution fails

            # Get coordinates for this sensor
            lat, lon = metadata.get_coordinates(city, street, country)
            
            if lat is None or lon is None:
                print(f"⚠️ Sensor {sensor_id}: cannot geocode location")
                failed += 1
                failed_sensors.append((sensor_id, "Geocoding failed"))
                continue

            # Clean and prepare air quality data 
            aq_df = cleaning.clean_and_append_data(
                aq_df_raw, sensor_id, 
                city=city, street=street, country=country,
                latitude=lat, longitude=lon, aqicn_url=working_feed_url
            )
            aq_df = aq_df.sort_values("date").drop_duplicates(subset=["date"], keep="first").reset_index(drop=True)
            
            # Add lagged and rolling features
            aq_df = feature_engineering.add_lagged_features(aq_df, "pm25", lags=[1,2,3])
            aq_df = feature_engineering.add_rolling_window_feature(aq_df, window_days=3, column="pm25", new_column="pm25_rolling_3d")
            
            # Calculate nearby sensor feature using location dict
            if len(sensor_locations) > 0:
                aq_df = feature_engineering.add_nearby_sensor_feature(
                    aq_df, 
                    sensor_locations,
                    n_closest=3
                )
            else:
                aq_df["pm25_nearby_avg"] = 0.0
            
            # Date range for weather
            end_date = aq_df["date"].max().date()
            start_date = end_date - timedelta(days=365 * 3)

            # Fetch weather
            weather_df = fetchers.get_historical_weather(
                sensor_id, start_date, end_date, lat, lon
            )
            
            if weather_df is None or len(weather_df) == 0:
                print(f"⚠️ No weather data for sensor {sensor_id}")
                failed += 1
                failed_sensors.append((sensor_id, "No weather data"))
                continue

            # Prepare weather data
            weather_df["date"] = pd.to_datetime(weather_df["date"]).dt.tz_localize(None)
            weather_df["sensor_id"] = int(sensor_id)
            weather_df = weather_df.astype({
                "sensor_id": "int32",
                "temperature_2m_mean": "float64",
                "precipitation_sum": "float64",
                "wind_speed_10m_max": "float64",
                "wind_direction_10m_dominant": "float64",
            })
            
            # Deduplicate weather by primary key before insert
            weather_df = weather_df.drop_duplicates(subset=["sensor_id", "date"], keep="first").reset_index(drop=True)
            
            # Insert with automatic materialization
            weather_fg.insert(weather_df)

            # Prepare air quality data
            aq_df["date"] = pd.to_datetime(aq_df["date"]).dt.tz_localize(None)
            aq_df = aq_df.astype({
                "sensor_id": "int32",
                "pm25": "float64",
                "pm25_lag_1d": "float64",
                "pm25_lag_2d": "float64",
                "pm25_lag_3d": "float64",
                "pm25_rolling_3d": "float64",
                "pm25_nearby_avg": "float64",
                "city": "string",
                "street": "string",
                "country": "string",
                "aqicn_url": "string",
                "latitude": "float64",
                "longitude": "float64",
            })
            
            # Final deduplication by primary key before insert
            aq_df = aq_df.drop_duplicates(subset=["sensor_id", "date"], keep="first").reset_index(drop=True)
            
            # Insert with automatic materialization
            air_quality_fg.insert(aq_df)

            existing_sensors.add(sensor_id)
            
            # Add this sensor's location to dict in format expected by feature_engineering
            sensor_locations[sensor_id] = {
                "latitude": lat,
                "longitude": lon,
                "city": city,
                "street": street,
                "country": country
            }
            
            successful += 1
            print(f"✅ Sensor {sensor_id} ({successful}/{remaining} complete)")

        except Exception as e:
            failed += 1
            failed_sensors.append((sensor_id, f"{type(e).__name__}: {str(e)[:100]}"))
            print(f"❌ Sensor {sensor_id}: {type(e).__name__}: {str(e)}")
            continue
    
    print(f"\n🎉 Backfill complete!")
    print(f"📊 Final Summary:")
    print(f"   ✅ Successfully processed: {successful}")
    print(f"   ❌ Failed: {failed}")
    print(f"   ⏩ Skipped (already processed): {skipped}")
    print(f"   📈 Total in feature store: {len(existing_sensors)}/{total_sensors}")

    if len(failed_sensors) > 0:
        print(f"\n⚠️  Failed Sensors Detail:")
        for sid, reason in failed_sensors:
            print(f"   • Sensor {sid}: {reason}")

else:
    print("\n✅ All sensors already processed. No backfill needed.")


## 1.5. Update Descriptions

In [ ]:
hopsworks_admin.update_air_quality_description(air_quality_fg)
hopsworks_admin.update_weather_description(weather_fg)

## 1.6. Add Validation to Feature Groups

In [ ]:
aq_expectation_suite = gx.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

# pm25 should be >= 0
aq_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "pm25",
            "min_value": 0.0,
            "max_value": None,
            "strict_min": False,
        },
    )
)

aq_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_in_type_list",
        kwargs={
            "column": "date",
            "type_list": ["datetime64", "Datetime", "Null"],
        },
    )
)


# sensor_id + date should be unique (PK)
aq_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_compound_columns_to_be_unique",
        kwargs={"column_list": ["sensor_id", "date"]},
    )
)

# rolling + lag features should be numeric (float or int)
for col in ["pm25_rolling_3d", "pm25_lag_1d", "pm25_lag_2d", "pm25_lag_3d"]:
    aq_expectation_suite.add_expectation(
        gx.core.ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_in_type_list",
            kwargs={
                "column": col,
                "type_list": ["float64", "Float64", "Int64", "Null"],
            },
        )
    )

aq_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_table_row_count_to_be_between",
        kwargs={"min_value": 1, "max_value": None}
    )
)

hopsworks_admin.save_or_replace_expectation_suite(air_quality_fg, aq_expectation_suite)


weather_expectation_suite = gx.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

weather_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(   
        expectation_type="expect_column_values_to_be_in_type_list",
        kwargs={
            "column": "date",
            "type_list": ["datetime64", "Datetime", "Null"],
        },
    )
)

# Temperature column - allow nulls, should be within physical range
weather_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "temperature_2m_mean",
            "min_value": -80,
            "max_value": 60,
            "mostly": 1.0,
        },
    )
)
weather_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_in_type_list",
        kwargs={
            "column": "temperature_2m_mean",
            "type_list": ["float64", "Float64", "Int64", "Null"],
        },
    )
)

# Precipitation column - should be >= 0, allow nulls
weather_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "precipitation_sum",
            "min_value": -0.1,
            "max_value": None,
            "mostly": 1.0,          # allow nulls
        },
    )
)
weather_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_in_type_list",
        kwargs={
            "column": "precipitation_sum",
            "type_list": ["float64", "Float64", "Int64", "Null"],
        },
    )
)

# Wind column - should be >= 0, allow nulls
weather_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "wind_speed_10m_max",
            "min_value": 0,
            "max_value": None,
            "mostly": 1.0,          # allow nulls
        },
    )
)
weather_expectation_suite.add_expectation(
    gx.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_in_type_list",
        kwargs={
            "column": "wind_speed_10m_max",
            "type_list": ["float64", "Float64", "Int64", "Null"],
        },
    )
)

gx.core.ExpectationConfiguration(
    expectation_type="expect_table_row_count_to_be_between",
    kwargs={"min_value": 1, "max_value": None}
)

hopsworks_admin.save_or_replace_expectation_suite(weather_fg, weather_expectation_suite)

## 1.7. Create Complete Feature View

In [ ]:
def create_feature_view(fs, air_quality_fg, weather_fg):
    # Select specific columns from weather_fg, excluding 'date' and 'sensor_id' since they're join keys
    weather_features = [f.name for f in weather_fg.features if f.name not in ['date', 'sensor_id']]
    
    query = (
        air_quality_fg.select_all()
        .join(weather_fg.select(weather_features), on=["sensor_id", "date"])
    )

    fv = fs.get_or_create_feature_view(
        name="air_quality_complete_fv",
        version=1,
        query=query,
        labels=["pm25"]
    )

    return fv

air_quality_fv = create_feature_view(fs, air_quality_fg, weather_fg)
print("\n✅ Feature view 'air_quality_complete_fv' created or updated successfully.")